In [1]:
import json
import openai
import datetime
from langame import LangameClient
c = LangameClient(path_to_config_file="../config.yaml")

In [2]:
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("generation")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: NtXdp7ODn4sR3YR2eEW6, garbage


Using 16 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 02/14/2022, 01:00:49
There are 793 embeddings of dim 768
	Compute estimated construction time of the index 02/14/2022, 01:00:49
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0000 secs
	Checking that your have enough memory available to create the index 02/14/2022, 01:00:49
2.8MB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0007 secs
	Selecting most promising index types given data characteristics 02/14/2022, 01:00:49
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 02/14/2022, 01:00:49
		-> Instanciate the index Flat 02/14/2022, 01:00:49
		>>> Finished "-> Instanciate the index Flat" in 0.0006 sec

100%|██████████| 1/1 [00:00<00:00, 449.69it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.0040 secs
	>>> Finished "Creating the index" in 0.0054 secs
	Computing best hyperparameters 02/14/2022, 01:00:49
	>>> Finished "Computing best hyperparameters" in 0.0000 secs
The best hyperparameters are: 
	Compute fast metrics 02/14/2022, 01:00:49


2379
	>>> Finished "Compute fast metrics" in 1.2243 secs
	Saving the index on local disk 02/14/2022, 01:00:50
	>>> Finished "Saving the index on local disk" in 0.0023 secs
Recap:
{'99p_search_speed_ms': 0.6217594631016252,
 'avg_search_speed_ms': 0.511996881208747,
 'compression ratio': 0.9999815281627787,
 'index_key': 'Flat',
 'index_param': '',
 'nb vectors': 793,
 'reconstruction error %': 0.0,
 'size in bytes': 2436141,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 1.2412 secs


In [3]:
out_file_name = f"../data/fine_tune_generation_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in memes:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e['topics'])} ###",
            "completion": f" {e['content']}\n",
        }, outfile)
        outfile.write('\n')

In [4]:
!head -n3 $out_file_name | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "ice breaker ###",
  "completion": " When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new?\n"
}
{
  "prompt": "ecology ###",
  "completion": " Have natural disasters gotten worse with the increase in human existence? If so, why?\n"
}
{
  "prompt": "ice breaker ###",
  "completion": " What would prompt you to make a dramatic change in your life. Is it worth it?\n"
}


In [11]:
!openai tools fine_tunes.prepare_data -f $out_file_name

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Analyzing...

- Your file contains 682 prompt-completion pairs
- There are 2 duplicated prompt-completion sets. These are rows: [408, 618]
- All prompts end with suffix ` ###`
- All completions end with suffix `\n`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 2 duplicate rows [Y/n]: ^C



In [5]:
# split to train and validation files
with open(out_file_name, "r") as infile:
    lines = infile.readlines()
    lines = lines[:int(len(lines) * 0.8)]
    with open(out_file_name.replace(".jsonl", "_train.jsonl"), "w") as outfile:
        outfile.writelines(lines)
    lines = lines[int(len(lines) * 0.8):]
    with open(out_file_name.replace(".jsonl", "_valid.jsonl"), "w") as outfile:
        outfile.writelines(lines)
        
train = openai.File.create(
  file=open(out_file_name.replace(".jsonl", "_train.jsonl")),
  purpose="fine-tune"
)
validation = openai.File.create(
  file=open(out_file_name.replace(".jsonl", "_valid.jsonl")),
  purpose="fine-tune"
)

ft = openai.FineTune.create(
    training_file=train["id"],
    validation_file=validation["id"],
    model="ada",
)

<FineTune fine-tune id=ft-h7BkxNXld3q10U3Rw4TN27po at 0x7fb8d7c59130> JSON: {
  "created_at": 1644449774,
  "events": [
    {
      "created_at": 1644449775,
      "level": "info",
      "message": "Created fine-tune: ft-h7BkxNXld3q10U3Rw4TN27po",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1
  },
  "id": "ft-h7BkxNXld3q10U3Rw4TN27po",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 76837,
      "created_at": 1644449772,
      "filename": "fine_tune_generation_10_02_2022_train.jsonl",
      "id": "file-yHUXl4Isc3vKYpBmBg8FkMKi",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 1644449774,
  "validation_files": 

In [ ]:
results = openai.FineTune.retrieve(
    ft["id"],
)
results